# Fine-tuning GPT Model

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key  = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=api_key)

### Upload file

In [2]:
training_file_name = "training_data.jsonl"

res = client.files.create(
  file=open(training_file_name, "rb"),
  purpose="fine-tune"
)
file_id = res.id

## Helper Function

In [13]:
def get_completion(prompt, model='gpt-4'):
    messages = [{'role': 'user', 'content': prompt}]
    response = client.chat.completions.create(
        model=model, 
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

### Create a fine-tuned model

In [3]:
ft_res = client.fine_tuning.jobs.create(
  training_file=file_id, 
  model="gpt-3.5-turbo"
)
print("Job ID:", ft_res.id)
print("Status:", ft_res.status)

Job ID: ftjob-WrjzKEVW8x6FHIWOyRRV82u4
Status: validating_files


In [4]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-WrjzKEVW8x6FHIWOyRRV82u4', created_at=1699974085, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-5DvmRlg7kGvkcggchTvEJDF0', result_files=[], status='validating_files', trained_tokens=None, training_file='file-cG6DhxnKEz2O4f66jWP8srDd', validation_file=None), FineTuningJob(id='ftjob-OIjGfoUk0nd8HC2Vt7nz9kt5', created_at=1699972109, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-5DvmRlg7kGvkcggchTvEJDF0', result_files=[], status='cancelled', trained_tokens=None, training_file='file-7hHkfu1u7LzfO4HCOzQ5M2Nn', validation_file=None), FineTuningJob(id='ftjob-r5B9uaRDRCxpHmX

In [7]:
# Retrieve the state of a fine-tune
response = client.fine_tuning.jobs.retrieve("ftjob-WrjzKEVW8x6FHIWOyRRV82u4")
print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-WrjzKEVW8x6FHIWOyRRV82u4
Status: succeeded
Trained Tokens: 4210


In [ ]:
# Cancel a job
client.fine_tuning.jobs.cancel("ftjob-OIjGfoUk0nd8HC2Vt7nz9kt5")


In [8]:
# List up to 50 events from a fine-tuning job
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-WrjzKEVW8x6FHIWOyRRV82u4", limit=50)
events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-WrjzKEVW8x6FHIWOyRRV82u4
Validating training file: file-cG6DhxnKEz2O4f66jWP8srDd
Files validated, moving job to queued state
Fine-tuning job started
Step 1/100: training loss=0.83
Step 11/100: training loss=2.84
Step 21/100: training loss=0.01
Step 31/100: training loss=1.19
Step 41/100: training loss=0.02
Step 51/100: training loss=0.00
Step 61/100: training loss=0.67
Step 71/100: training loss=0.00
Step 81/100: training loss=0.22
Step 91/100: training loss=0.00
New fine-tuned model created: ft:gpt-3.5-turbo-0613:personal::8KqXmPZh
The job has successfully completed


In [9]:
# Retrieve the state of a fine-tune
response = client.fine_tuning.jobs.retrieve("ftjob-WrjzKEVW8x6FHIWOyRRV82u4")
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0613:personal::8KqXmPZh


In [ ]:
# Delete a fine-tuned model (must be an owner of the org the model was created in)
# client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

### Use your fine-tune model

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [19]:
fine_tuned_model = "ft:gpt-3.5-turbo-0613:personal::8KqXmPZh"

new_prompt = "Is it legal to drive a car shirtless in Thailand?"
response = get_completion(new_prompt, fine_tuned_model)
print(response)

new_prompt = "Who is the priminister of Thailand?"
response = get_completion(new_prompt, fine_tuned_model)
print(response)

new_prompt = "How many temples are in Thailand?"
response = get_completion(new_prompt, fine_tuned_model)
print(response)

No, it is not legal to drive a car shirtless in Thailand.
The Prime Minister of Thailand is Prayuth Chan-o-cha.
There are approximately 35,000 temples in Thailand.
